# Get trendy tickers' company information from Yahoo Finance api + ESG api
check trending tickets across market regions, sort by top mktprice, start from specified top X tickers. uplimit = 20

region = ['US', 'BR', 'AU', 'CA', 'FR', 'DE', 'HK', 'IN', 'IT', 'ES', 'GB', 'SG']
Yahoo finance API: https://rapidapi.com/apidojo/api/yh-finance/
ESG API document: https://www.esgenterprise.com/esg-enterprise-data-api-services/
both API requires api key that could be required through signing in websites.

I make general search of those trendy tickers by discarding usd, cad and etc terms from trendy tickers search results. For example, shib-usd and shib-cad are all searched with 'shib' since the esg api only returns the origin ticker symbol.
Also, due to the rate limit of esg api, I set trendy tickers search results limited in 5 tickers. but users can start at any point of trendy tickers list sorted by market prices, the default is start from the 5th trendy tickers and return 5-10 trendy tickers esg scores.

skills:
Data Acquisition:
Import data from json.
Use APIs to obtain data. - esg api + yahoo finance api
Write an API client for an API and/or functions associated with API interaction.
Handle, parse, and transform JSON and XML.
Data Cleaning, Transformation, and Organization:
Use data wrangling to transform data into a dataframe ready for analysis.
Use loops processes.
Use functions and functional programming to export repetitive or difficult tasks.
Handle and process strings and use regular expressions.
Documentation and Presentation:
Provide and document useful functions as part of a Python package.
capture error
Use Jupyter Notebook to generate a vignette on how to use your package functions.

In [49]:
import requests
from requests.exceptions import HTTPError
import requests.auth
import re
import pandas as pd
import os
regionlist = ['US', 'BR', 'AU', 'CA', 'FR', 'DE', 'HK', 'IN', 'IT', 'ES', 'GB', 'SG']
YAHOOF_KEY = os.getenv('YAHOOF_KEY')
app_key = os.getenv('ESG_KEY')

def trendyesg(region, start=5):
    """
    Get region's trendy tickers' esg score based on given region and regular rank of market price.

    Parameters
    ----------
    region : string
      A string that limit the region of market consumers are interested in, which include US, BR, AU, CA, FR, DE, HK, IN, IT, ES, GB, and SG.
    start : int
      A int that indicate the ranking place to start from.

    Returns
    -------
    pandas.dataframe
      The dataframe of trendy tickers' esg, environmental, social,governance score, and its ticker based on consumers interested region and the lowest ranking consumers want to start to look at

    Examples
    --------
    >>> from brandaniethical import trendyesg 
    >>> trendyesg(region='US',start =8)
	company_name	env_score	soc_score	gov_score	total	ticker
0	Medtronic plc	500	304	310	1114	MDT
1	Pfizer Inc.	510	300	305	1115	PFE
2	Roblox Corporation	245	304	220	769	RBLX
3	Lemonade, Inc.	520	303	300	1123	LMND
4	Nucor Corporation	251	326	210	787	NUE
    """
    assert region in regionlist, f"This function only works on {regionlist} but region={region}."
    assert isinstance(start, int), f"This function only works on int but start={start}."
    def get_trendycom(region, start=5):
        url = "https://yh-finance.p.rapidapi.com/market/get-trending-tickers"
        def trendytick(region):
            querystring = {"region":region}
            try:
                headers = {
                    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
                    'x-rapidapi-key': YAHOOF_KEY
                            }
                response = requests.request("GET", url, headers=headers, params=querystring)
                response.raise_for_status()

            except HTTPError as http_err:
                print(f'HTTP error occurred: {http_err}')
            except Exception as err: 
                print(f'Other error occurred: {err}')
            return response.json()

        cominfolist = trendytick(region)

        region = []
        shortName = []
        regularMarketPrice = []
        symbol=[]
        trendtick_df = pd.DataFrame()
    
        for i in cominfolist['finance']['result'][0]['quotes']:   
            shortName.append(i['shortName'])
            regularMarketPrice.append(i['regularMarketPrice'])
            symbol.append(i['symbol'])

        trendtick_df['shortName'] = pd.Series(shortName)
        trendtick_df['regularMarketPrice'] = pd.Series(regularMarketPrice)
        trendtick_df['symbol'] = pd.Series(symbol)
        trendtick_df['maxprice_rank'] = trendtick_df['regularMarketPrice'].rank(method='max')

        return trendtick_df.sort_values(by=['maxprice_rank']).iloc[start:start+5]
    
    #get trendy tickers outcome
    tdf = get_trendycom(region=region, start=5)
    assert tdf.empty == False, f'there is no search results of {region} market'
    def ticktrans(tdf):
        #make fuzzy search of ticker e.g.
        ttlist = [i for i in tdf['symbol']]
        if region == 'US':
            reg_tick = '(^.+?)-|(^.*)'
            for tick in ttlist:
                try:
                    ticklist = [re.match(reg_tick,tick).group(1) for tick in ttlist]
                    ticklist.extend([re.match(reg_tick,tick).group(2) for tick in ttlist])
                    ticklist = list(filter(None, ticklist))
                    return ticklist
                except TypeError as e:
                    print(f'{e}: the ticker is not supported')
        else:
            return ttlist
    ticktransl = ticktrans(tdf)
    
    base_url = 'https://tf689y3hbj.execute-api.us-east-1.amazonaws.com/prod/authorization/'
    df = pd.DataFrame()
    company_name=[]
    env_score=[]
    soc_score=[]
    gov_score=[]
    total=[]
    region=[]
    ticker=[]
    for brand_tick in ticktransl:
        try:
            response_esg = requests.get(base_url+'search?',params={'q':brand_tick,'token':app_key})
            response_esg.raise_for_status()
            company_name.append(response_esg.json()[0]['company_name'])
            env_score.append(response_esg.json()[0]['environment_score'])
            soc_score.append(response_esg.json()[0]['social_score'])
            gov_score.append(response_esg.json()[0]['governance_score'])
            total.append(response_esg.json()[0]['total'])
            ticker.append(brand_tick)
        except (IndexError, TypeError) as error:
            company_name.append(brand_tick)
            env_score.append('NA')
            soc_score.append('NA')
            gov_score.append('NA')
            total.append('NA')
            ticker.append(brand_tick)
            print(f"{brand_tick}'s esg not in database")
        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')
        except Exception as err: 
            print(f'Other error occurred: {err}')
    df['company_name'] = pd.Series(company_name)
    df['env_score'] = pd.Series(env_score)
    df['soc_score'] = pd.Series(soc_score)
    df['gov_score'] = pd.Series(gov_score)
    df['total'] = pd.Series(total)
    df['ticker'] = pd.Series(ticker)
    return df

In [6]:
#show case
trendyesg(region='US',start =8)

,company_name,env_score,soc_score,gov_score,total,ticker
0,Medtronic plc,500,304,310,1114,MDT
1,Pfizer Inc.,510,300,305,1115,PFE
2,Roblox Corporation,245,304,220,769,RBLX
3,"Lemonade, Inc.",520,303,300,1123,LMND
4,Nucor Corporation,251,326,210,787,NUE


In [50]:
trendyesg(region='US',start =10)

LCID's esg not in database


,company_name,env_score,soc_score,gov_score,total,ticker
0,"AMC Entertainment Holdings, Inc.",210,216,200,626,AMC
1,LCID,NA,NA,NA,NA,LCID
2,General Motors Company,391,323,215,929,GM
3,"Bottomline Technologies (de), Inc.",540,302,300,1142,EPAY
4,Cerner Corporation,532,300,300,1132,CERN
